In [1]:
#####Adaptive Moment estimation#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
####データの発生####
##データの設定
segment = 5
hh = 10000   #サンプル数
pt = numpy.random.poisson(numpy.random.gamma(5.0, 1/0.5, hh))   #購買機会数
pt[pt==0] = 1   #購買機会が0なら1に置き換え
hhpt = np.sum(pt)   #総サンプル数
member = 10   #選択可能数
k = 5   #説明変数数

In [3]:
##idとセグメントの設定
#idの設定
id = np.repeat(range(hhpt), member)
no = np.zeros(hhpt*member, dtype='int')
start = 0
for i in range(hhpt):
    no[start:start+member] = np.array(range(member))
    start += member

user_id = np.repeat(range(hh), pt)
pt_id = np.zeros(hhpt, dtype='int')
start = 0
for i in range(hh):
    pt_id[start:start+pt[i]] = np.array(range(pt[i]))
    start += pt[i]
u_id = np.repeat(user_id, member)
user_dt = sparse.coo_matrix((np.repeat(1, hhpt), (range(hhpt), user_id)), shape=(hhpt, hh)).T

In [4]:
#インデックスを設定
index = np.array(range(hhpt*member))
user_list = [i for i in range(hh)]
id_list = [i for i in range(hhpt)]
no_list = [i for i in range(member)]

for i in range(hh):
    user_list[i] = index[u_id==i]
for i in range(hhpt):
    id_list[i] = index[id==i]
for j in range(member):
    no_list[j] = index[no==j]

In [5]:
#セグメントの設定
theta = thetat = np.random.dirichlet(np.repeat(3.0, segment), 1).reshape(-1)
Z = np.random.multinomial(1, theta, hh)
seg = np.dot(Z, range(segment))

In [6]:
##説明変数の発生
#切片の設定
diag_ind = np.ravel(np.diag(np.ravel(np.matlib.repmat(1, member, 1))))
diag_vec = np.ravel(np.matlib.repmat(diag_ind, hhpt, 1))
intercept = np.reshape(diag_vec, (hhpt*member, member))[:, range(member-1)]

In [7]:
#衣装の割当を設定
c_num = 8
prob = np.random.dirichlet(np.repeat(1.5, c_num), member); m = np.argmin(np.sum(prob, axis=0))
Cloth = np.zeros((hhpt*member, c_num-1))
for j in range(member):
    x = np.random.multinomial(1, prob[j, ], hhpt)
    Cloth[no_list[j], :] = np.delete(x, m, axis=1)

In [8]:
#どのメンバーの勧誘回だったか
prob = np.repeat(1/member, member)
scout = np.zeros((hhpt, member))
for i in range(hhpt):
    while True:
        x = np.random.multinomial(2, prob, 1).reshape(-1)
        if np.max(x) == 1:
            break
    scout[i, :] = x
Scout = scout.reshape(-1)

In [9]:
#レベルの対数
lv_weibull = np.round(scipy.stats.weibull_min.rvs(1.8, scale=250, size=hh*member))
lv = np.log(np.random.choice(lv_weibull[lv_weibull > 80], hh, replace=False))
Lv = np.zeros((hhpt*member, member-1))
for i in range(hh):
    Lv[user_list[i], :] = (lv[i] * np.identity(member))[:, range(member-1)][np.tile(range(member), pt[i]), :]

In [10]:
#スコアの対数
score = np.abs(np.random.normal(0, 0.75, hhpt))
Score = np.zeros((hhpt*member, member-1))
for i in range(hh):
    Score[user_list[i], :] = (score[i] * np.identity(member))[:, range(member-1)][np.tile(range(member), pt[i]), :]

In [11]:
#データの結合
Data = np.hstack((intercept, Cloth, Scout.reshape((hhpt*member, 1)), Lv, Score))
sparse_data = scipy.sparse.csr_matrix(Data)
k1 = intercept.shape[1]; k2 = Cloth.shape[1]; k3 = 1; k4 = Lv.shape[1]; k5 = Score.shape[1]
k = Data.shape[1]
pd.DataFrame(np.round(Data, 3))   #データを確認

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.134,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.134,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.134,0.000,0.000,0.000,0.000,0.000,0.000
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.134,0.000,0.000,0.000,0.000,0.000
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.134,0.000,0.000,0.000,0.000
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.134,0.000,0.000,0.000
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.134,0.000,0.000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.134,0.000
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.403,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.134
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [12]:
##パラメータの設定
beta1 = beta4 = beta5 = np.zeros((segment, member-1))
beta2 = np.zeros((segment, c_num-1))
beta3 = np.repeat(0.0, segment)

for j in range(segment):
    beta1[j, :] = np.random.uniform(-1.0, 4.0, member-1)   #切片の回帰係数
    beta2[j, ] = np.random.uniform(-2.0, 3.0, c_num-1)   #衣装の回帰係数
    beta3[j] = np.random.uniform(0.6, 4.0, 1)   #勧誘の回帰係数
    beta4[j, :] = np.random.uniform(-0.4, 0.4, member-1)   #レベルの回帰係数
    beta5[j, :] = np.random.uniform(-0.6, 0.6, member-1)   #スコアの回帰係数
beta = betat = np.hstack((beta1, beta2, beta3.reshape((segment, 1)), beta4, beta5)).T   #回帰係数を結合
b = beta.reshape(-1)

In [13]:
##応答変数の発生
#ロジットと確率を発生
U = np.sum(np.dot(Data, beta) * Z[u_id, ], axis=1).reshape(hhpt, member)
Pr = np.exp(U) / np.sum(np.exp(U), axis=1).reshape(hhpt, 1)

In [14]:
#応答変数の発生
y = np.array([np.random.multinomial(1, Pr[i, :], 1) for i in range(hhpt)]).reshape((hhpt, member))
y_vec = y.reshape(-1)
np.sum(y, axis=0)

array([ 2613, 16885,  7882, 14615, 15670,  4543, 10229,  5484, 20598,
        2346])

In [15]:
####EMアルゴリズムで有限混合ロジットモデルを推定####
##完全データのロジットモデルの対数尤度
def cll(b, y, y_vec, Data, zpt, hhpt, member, k, segment, u_id):
    #パラメータの設定
    beta = b.reshape(k, segment)

    #潜在変数での重み付き確率
    U = np.exp(np.sum(np.dot(Data, beta) * zpt[u_id, ], axis=1).reshape(hhpt, member))
    Pr = U / np.sum(U, axis=1).reshape(hhpt, 1)

    #完全データの対数尤度の和
    LL = -np.sum(np.dot((y * np.log(Pr)), np.repeat(1, member)))
    return(LL)

In [16]:
##完全データのロジットモデルの対数微分関数
def dll(b, y, y_vec, Data, zpt, hhpt, member, k, segment, u_id):
    #パラメータの設定
    beta = b.reshape(k, segment)

    #潜在変数での重み付き確率
    U = np.exp(np.dot(Data, beta))
    dlogit = np.zeros((segment, k))

    for j in range(segment):
        #効用と確率を設定
        u = U[:, j].reshape(hhpt, member)
        Pr = u / np.sum(u, axis=1).reshape(hhpt, 1)

        #勾配ベクトルを定義
        Pr_vec = Pr.reshape(-1)
        dlogit[j, :] = np.sum(zpt[u_id, j].reshape(hhpt*member, 1) * (y_vec - Pr_vec).reshape(hhpt*member, 1) * Data, axis=0)

    LLd = -dlogit.T.reshape(-1)
    return(LLd)

In [17]:
##観測データでの尤度と潜在変数zを計算する関数
def ollz(beta, y, Data, zpt, hhpt, member, k, segment, user_dt):
    #潜在変数ごとの確率と尤度を設定
    U = np.exp(np.dot(Data, beta))
    LLho = np.zeros((hh, segment))

    for j in range(segment):
        #効用と確率を設定
        u = U[:, j].reshape(hhpt, member)
        Pr = u / np.sum(u, axis=1).reshape(hhpt, 1)

        #ユーザーごとの尤度を取る
        LLho[:, j] = np.exp((scipy.dot(user_dt, sparse.csr_matrix(np.dot(y * np.log(Pr), np.repeat(1, member))).T).todense())).reshape(-1)

    #観測データの対数尤度
    r = np.tile(theta, hh).reshape(hh, segment)
    LLobz = np.sum(np.log(np.dot(r * LLho, np.repeat(1, segment))))

    #潜在変数zの推定
    LLi = r * LLho
    z = LLi / np.dot(LLi, np.repeat(1, segment)).reshape(hh, 1)
    return LLobz, z

In [18]:
##EMアルゴリズムの初期値を設定
#混合率と潜在変数の初期値
theta = np.repeat(1/segment, segment)
zpt = np.tile(theta, hh).reshape(hh, segment)

#パラメータの初期値
b = np.random.uniform(-0.25, 0.25, segment*k)
beta = b.reshape(k, segment)

In [19]:
##観測データの対数尤度と潜在変数
oll = ollz(beta, y, Data, zpt, hhpt, member, k, segment, user_dt)
zpt = oll[1]
LL1 = oll[0]

In [20]:
##アルゴリズムの設定
#EMアルゴリズムの設定
iter = 0
rp = 200   #最大繰り返し数
LL = -1000000000   #対数尤度の初期値
dl1 = 100   #EMアルゴリズムの対数尤度の差の初期値
tol1 = 1.0

#ADAMのパラメータを設定
alpha = 0.05
moment1 = 0.0
moment2 = 0.0
weight1 = 0.75
weight2 = 0.99
epsilon = np.power(10.0, -8)
LLho1 = -10000000
dl2 = 100
tol2 = 1.0

In [21]:
##EMアルゴリズムでパラメータを推定
while abs(dl1) >= tol1:
    
    #完全データのロジットモデルをADAMで推定(Mステップ)
    dl2 = 100
    while abs(dl2) >= tol2:
        g = dll(b, y, y_vec, Data, zpt, hhpt, member, k, segment, u_id)  #パラメータの勾配
        moment1 = weight1*moment1 + (1-weight1)*g
        moment2 = weight2*moment2 + (1-weight2)*np.power(g, 2)
        moment_hat1 = moment1 / (1-weight1) 
        moment_hat2 = moment2 / (1-weight2)
        b = b - alpha*moment_hat1/(np.sqrt(moment_hat2) + epsilon)   #パラメータの更新

        #完全データの対数尤度を更新
        LLho = -cll(b, y, y_vec, Data, zpt, hhpt, member, k, segment, u_id)
        dl2 = LLho - LLho1
        LLho1 = LLho
        
    #混合率を更新
    beta = b.reshape(k, segment)   
    theta = np.sum(zpt, axis=0) / hh   
    
    #Eステップで対数尤度の期待値と潜在変数を更新(Eステップ)
    obsllz = ollz(beta, y, Data, zpt, hhpt, member, k, segment, user_dt)
    LL = obsllz[0]
    zpt = obsllz[1]

    #EMアルゴリズムのパラメータを更新
    iter = iter + 1
    dl1 = LL - LL1
    LL1 = LL
    print(LL)

-135105.67973956422
-117806.38552015179
-112545.35292920707
-111964.79123692543
-111958.41885517543
-111957.53790685639
